<h1 align="center">Zewail University of Science and Technology</h1>
<h2 align="center">CIE 417 (Fall 2018)</h2>
<h2 align="center">Lab 5: Cross Validation and Decision Trees</h3>
<h3 align="center">18/10/2018</h3>

## <font color = "#b1b100">Cross Validation <font/>
<a href="https://dziganto.github.io/cross-validation/data%20science/machine%20learning/model%20tuning/python/Model-Tuning-with-Validation-and-Cross-Validation/">This link<a/> <font color = "#000000">provides a good tutorial on bias variance tradeoff and validation/cross-validation<font/>

<img src="bias-variance-tradeoff.png">
source: https://dziganto.github.io/cross-validation/data%20science/machine%20learning/model%20tuning/python/Model-Tuning-with-Validation-and-Cross-Validation/

## <font color = "#af00af"> 1) How can we tune hyperparameters if the dataset is small that we can't afford creating a separate validation set? <font/>
<img src="cross_validation.gif">
source: https://imada.sdu.dk/~marco/Teaching/AY2010-2011/DM825/

### K-fold cross validation (3-fold)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold

X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
kf = KFold(n_splits=3)
print ("Original Data: ", X)
print ("Number of Splits: ", kf.get_n_splits(X))
for train_index, test_index in kf.split(X):
    print("TRAIN:", X[train_index], "TEST:", X[test_index])

### Leave One Out (LOO) cross validation (n-fold)

In [ ]:
from sklearn.model_selection import LeaveOneOut

X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
print ("Original Data: ", X)
loo = LeaveOneOut()
print ("Number of Splits: ", loo.get_n_splits(X))
for train_index, test_index in loo.split(X):
    print("TRAIN:", X[train_index], "TEST:", X[test_index])

### Leave P Out (LPO) cross validation (Leave 2 Out) 
Creates overlapping folds, unlike kfold method

In [ ]:
from sklearn.model_selection import LeavePOut

X = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
print ("Original Data: ", X)
loo = LeavePOut(p = 2)
print ("Number of Splits: ", loo.get_n_splits(X))
for train_index, test_index in loo.split(X):
    print("TRAIN:", X[train_index], "TEST:", X[test_index])

### Rules of thumb: 
1. generally, use 5-fold or 10-fold cross-validation
2. avoid LOO and LPO for large datasets, as they are very costly
3. as the dataset gets larger, decrease the number of folds (3-fold, 2-fold, etc)
4. if the dataset is very large, avoid cross validation altogether and just dedicate a separate validation set

## What is the Boston dataset?

##### Attributes:
1. crim: 
per capita crime rate by town.

2. zn: 
proportion of residential land zoned for lots over 25,000 sq.ft.

3. indus: 
proportion of non-retail business acres per town.

4. chas: 
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

5. nox: 
nitrogen oxides concentration (parts per 10 million).

6. rm: 
average number of rooms per dwelling.

7. age: 
proportion of owner-occupied units built prior to 1940.

8. dis: 
weighted mean of distances to five Boston employment centres.

9. rad: 
index of accessibility to radial highways.

10. tax: 
full-value property-tax rate per \$10,000.

11. ptratio: 
pupil-teacher ratio by town.

12. black: 
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

13. lstat: 
lower status of the population (percent).

##### Target:
median value of owner-occupied homes in \$1000s.

### Load and Split Dataset

In [185]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# import Boston dataset
boston = datasets.load_boston()

X = boston.data
y = boston.target

#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 0)

del X, y
print (f"training set size: {X_train.shape[0]} samples \ntest set size: {X_test.shape[0]} samples")

training set size: 404 samples 
test set size: 102 samples


### Preprocess Dataset

In [186]:
from sklearn.preprocessing import StandardScaler

#Standardize Dataset (substract training set mean and divide by training set standard deviation)
scaler = StandardScaler().fit(X_train) #the scaler is fitted to the training set (it gets the mean and std of the training set)
X_train_standardized = scaler.transform(X_train) #the scaler is applied to the training set
X_test_standardized = scaler.transform(X_test) #the scaler is applied to the test set

del X_train, X_test

### <font color="#ef0000"> Exercise 1: Train using scikit-learn Ridge, Use 5-fold cross-validation for the choice of the regularization parameter (alpha) using scikit-learn cross_validate) <font/>
#### <font color="#ef0000"> Write your code here <font/>
<font color="#ef0000"> use neg_mean_squared_error as the scoring parameter, print the training error and the cross validation error for each alpha <font/>

In [187]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

alphas = [1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
k = 5

## write your code here, print the training error and the cross validation error for each alpha
for alpha in alphas:
    classifier = Ridge(alpha=alpha, fit_intercept=True)
    result = cross_validate(X=X_train_standardized, y=y_train, cv=k, estimator=classifier, return_train_score =True, 
                            scoring="neg_mean_squared_error")
    fit_time = np.mean(result['fit_time'])
    score_time = np.mean(result['score_time'])
    train_score = -np.mean(result['train_score'])
    validation_Score = -np.mean(result['test_score'])
    print (f"alpha = {alpha},\ttraining time: {fit_time:.3f}s, scoring time: {score_time:.3f}s, \
training score: {train_score:.2f}, validation score: {validation_Score:.2f}")

alpha = 0.01,	training time: 0.079s, scoring time: 0.001s, training score: 19.13, validation score: 21.18
alpha = 0.1,	training time: 0.001s, scoring time: 0.000s, training score: 19.13, validation score: 21.17
alpha = 1.0,	training time: 0.001s, scoring time: 0.000s, training score: 19.13, validation score: 21.16
alpha = 10.0,	training time: 0.001s, scoring time: 0.000s, training score: 19.25, validation score: 21.14
alpha = 100.0,	training time: 0.001s, scoring time: 0.000s, training score: 21.51, validation score: 22.96
alpha = 1000.0,	training time: 0.001s, scoring time: 0.000s, training score: 39.13, validation score: 40.11


## <font color = "#a15151">Decision Trees <font/>


<img src="restaurant-tree.png">
<br/> 
<br/> 
source: Artificial Intelligence A Modern Approach by Stuart Russell and Peter Norvig

### <font color = "#af00af"> What if there is a continuous variable? <font/>

<img src="regression_tree.png">

source: Elements of Statistical Learning by Trevor Hastie, Robert Tibshirani and Jerome Friedman

### <font color = "#af00af"> How can we obtain a continuous output (Regression Tree)? <font/>

<img src="tree_regression.png">

source: http://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html

### <font color = "#af00af"> Which is the better choice for a node, and why? <font/>

<img src="restaurant-stub.png">
<br/> 
<br/> 
source: Artificial Intelligence A Modern Approach by Stuart Russell and Peter Norvig

### <font color = "#af00af"> Why don't we try all tree combinations and find the optimum tree? <font/>

### Entropy
#### Entropy represents the amount of randomness, the more we know (have information), the less the entropy becomes

<img src="EntropyVersusProbability.png">

source: http://matlabdatamining.blogspot.com/2006/11/introduction-to-entropy.html

#### We recursively choose the node that reduces the entropy (the node with the maximum Information Gain)

$$Entropy\ H(\pi) = -\sum \pi log_2(\pi)$$

$$Two\ Class\ Entropy\ H(\frac{p}{p+n},\frac{n}{p+n}) = -\frac{p}{p+n} log_2(\frac{p}{p+n}) -\frac{n}{p+n} log_2(\frac{n}{p+n})$$

$$Expected\ Entropy\ after\ adding\ node\ A\ EH(A)=\sum_{i=1}^k probability\ of\ leaf\ i\ X\ entropy\ of\ leaf\ i$$

$$EH(A)= \sum_{i=1}^k \frac{p_i+n_i}{p+n} H(\frac{p_i}{p+n},\frac{n_i}{p+n}) $$

$$Information\ Gain\ I(A)= H(\frac{p}{p+n},\frac{n}{p+n}) - EH(A)$$

### <font color = "#af00af"> Does this greedy algorithm guarantees getting the optimum tree? <font/>

### <font color = "#af00af"> When to stop creating more nodes? <font/>
* #### If all the remaining examples are all positive or negative
* #### If there are no examples left for this certain case (return the dominant class of the parent node)
* #### If there are no more attributes (return the dominant class of the remaining examples)


### <font color = "#ef0000"> Exercise 2: Determine (by hand) the first 2 nodes of a decision tree for this data<font/>

In [122]:
import pandas as pd

Patrons = ["Some", "Full", "Some", "Full", "Full", "Some", "None", "Some", "Full", "Full", "None", "Full"]
Type = ["French", "Thai", "Burger", "Thai", "French", "Italian", "Burger", "Thai", "Burger", "Italian", "Thai", "Burger"]
Hungry = [True, True, False, True, False, True, False, True, False, True, False, True]
WillWait = [True, False, True, True, False, True, False, True, False, False, False, True]
df2 = pd.DataFrame({'Patrons': Patrons, "Type": Type, "Hungry": Hungry, "WillWait (y)": WillWait})
df2

,Patrons,Type,Hungry,WillWait (y)
0,Some,French,True,True
1,Full,Thai,True,False
2,Some,Burger,False,True
3,Full,Thai,True,True
4,Full,French,False,False
5,Some,Italian,True,True
6,None,Burger,False,False
7,Some,Thai,True,True
8,Full,Burger,False,False
9,Full,Italian,True,False


### <font color="#ef0000"> Exercise 3: Classify the iris dataset using scikit-learn DecisionTreeClassifier <font/>

In [176]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# import iris dataset
iris = datasets.load_iris()

# We would use only the first two features
X = iris.data
y = iris.target

#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state = 1)

del X, y
print (f"training set size: {X_train.shape[0]} samples\ntest set size: {X_test.shape[0]} samples")

training set size: 105 samples
test set size: 45 samples


#### <font color="#ef0000"> Write Your Code Here <font/>
<font color="#ef0000"> use entropy as the criterion parameter, print the training and testing accuracy <font/>

In [177]:
from sklearn.tree import DecisionTreeClassifier

## write your code here
classifier = DecisionTreeClassifier(criterion="entropy")
classifier.fit(X_train, y_train)
train_accuracy = classifier.score(X_train, y_train)
test_accuracy = classifier.score(X_test, y_test)
print(f"Training Accuracy: {train_accuracy*100:.2f}, Testing Accuracy: {test_accuracy*100:.2f}")

Training Accuracy: 100.00, Testing Accuracy: 95.56


### <font color="#ef0000"> Exercise 4: Predict the Boston dataset used in Exercise 1 using scikit-learn DecisionTreeRegressor <font/>

In [182]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

# import Boston dataset
boston = datasets.load_boston()

X = boston.data
y = boston.target

#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 0)

del X, y
print (f"training set size: {X_train.shape[0]} samples \ntest set size: {X_test.shape[0]} samples")

training set size: 404 samples 
test set size: 102 samples


#### <font color="#ef0000"> Write Your Code Here <font/>


In [183]:
from sklearn.tree import DecisionTreeRegressor

## write your code here
classifier = DecisionTreeRegressor()
classifier.fit(X_train, y_train)
train_accuracy = classifier.score(X_train, y_train)
test_accuracy = classifier.score(X_test, y_test)
print(f"Training Accuracy: {train_accuracy*100:.2f}, Testing Accuracy: {test_accuracy*100:.2f}")

Training Accuracy: 100.00, Testing Accuracy: 56.23


### <font color = "#af00af"> Advantages of Decision Trees? <font/>
* #### Interpretability of Results
* #### Powerful nonparametric model
* #### It has no problem whether the inputs and outputs are binary, categorical or continuous

### <font color = "#af00af"> Problems of Decision Trees? <font/>
* #### Overfitting
* #### Very High Variance

## <font color = "#ff7777"> Try to solve Assignment 2 before next midterm (It will be included in the exam) <font/>